**Домашнее задание**

ПАКЕТ SURPRISE

Используйте данные MovieLens 1M https://grouplens.org/datasets/movielens/latest/.

Можно использовать любые модели из пакета.

Получите RMSE на тестовом сете 0.87 и ниже.

Комментарий преподавателя:

В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [1]:
from surprise import KNNWithMeans, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate

import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Объединим датасеты по признаку movieId и отбросим строки, для которых нет рейтинга.

In [5]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

Посмотрим чего больше: фильмов или пользователей.

In [6]:
print(f"Всего фильмов: {len(movies_with_ratings['movieId'].unique())}.")
print(f"Всего пользователей: {len(movies_with_ratings['userId'].unique())}.")

Всего фильмов: 9724.
Всего пользователей: 610.


Посмотрим, насколько будет разряженной матрица оценок.

In [7]:
rt = pd.DataFrame(movies_with_ratings.groupby("userId")["movieId"].count()).reset_index()
rt.columns=["userId", "count"]
rt.describe()

,userId,count
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


Как видим, при 9724 фильмах оценок в среднем 165, по 50-ой перцентили лишь 70 с небольшим. Матрица получается довольно разряженной.

Создадим датасет для пакета surprise.

In [8]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [9]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


Подготовим данные для использования их моделями.

In [10]:
reader = Reader(rating_scale=(ratings["rating"].min(), ratings["rating"].max()))
data = Dataset.load_from_df(dataset, reader)

Для модели KNNWithMeans попробуем оба варианта фильтрации (user based и item based) и вычислим RMSE еа основе кроссвалидации по 5 фолдам.

In [11]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
result = cross_validate(algo, data, measures=['RMSE'], cv=5)
print(f"RMSE: {np.mean(result['test_rmse'])}.")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8965524731719032.


In [12]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
result = cross_validate(algo, data, measures=['RMSE'], cv=5)
print(f"RMSE: {np.mean(result['test_rmse'])}.")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8819788684213743.


Оба варианта оказались недостаточно точными. Попробуем все то же самое, но с моделью KNNBaseline.

In [13]:
algo = KNNBaseline(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
result = cross_validate(algo, data, measures=['RMSE'], cv=5)
print(f"RMSE: {np.mean(result['test_rmse'])}.")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8777092379664895.


In [14]:
algo = KNNBaseline(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
result = cross_validate(algo, data, measures=['RMSE'], cv=5)
print(f"RMSE: {np.mean(result['test_rmse'])}.")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8532539697130529.
